<a href="https://colab.research.google.com/github/Chawlaji8781/M-G-ACFSVM/blob/main/M%26G_ACFSVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cvxopt
import cvxopt.solvers

In [2]:
def linear_kernel(x1,x2):
  return np.dot(x1, x2)

def polynomial_kernel(x1,x2,p=3):
  return (1 + np.dot(x1,x2))**p

def gaussian_kernel(x,y,sigma=5.0):
  return np.exp((-np.linalg.norm(x-y)**2) * (sigma**2))
  # np.exp(-np.linalg.norm(x-y)**2 / (2*(sigma**2)))

# Support Vector Machine

In [3]:
class SVM():
  def __init__(self, kernel = linear_kernel, C = 1):
    self.kernel = kernel
    self.C = C

  def fit(self,X,y):
    n_samples, n_features = X.shape

    #Gram Matrix
    K = np.zeros((n_samples,n_samples))
    for i in range(n_samples):
      for j in range(n_samples):
        K[i,j] = self.kernel(X[i], X[j])

    P = cvxopt.matrix(np.outer(y,y) * K, tc='d')
    q = cvxopt.matrix(np.ones(n_samples) * -1, tc='d')
    A = cvxopt.matrix(y, (1,n_samples), tc='d')
    b = cvxopt.matrix(0.0)
    
    if self.C is None:
      G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1), tc='d')
      h = cvxopt.matrix(np.zeros(n_samples), tc='d')
    else:
      tmp1 = np.diag(np.ones(n_samples) * -1)
      tmp2 = np.identity(n_samples)
      G = cvxopt.matrix(np.vstack((tmp1, tmp2)), tc='d')
      tmp1 = np.zeros(n_samples)
      tmp2 = np.ones(n_samples) * self.C
      h = cvxopt.matrix(np.hstack((tmp1, tmp2)), tc='d')

    # solve QP problem
    solution = cvxopt.solvers.qp(P, q, G, h, A, b, verbose=True)

    #Lagrange Multipliers:
    a = np.ravel(solution['x'])

    #For support vectors a != 0
    sv = a > 1e-7
    ind = np.arange(len(a))[sv]
    self.a = a[sv]
    self.sv = X[sv]
    self.sv_y = y[sv]
    print ("%d support vectors out of %d points" % (len(self.a), n_samples))

    #Intercept
    self.b = 0
    for n in np.arange(len(self.a)):
      self.b += self.sv_y[n]
      self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
    self.b /= len(self.a)

    if self.kernel == linear_kernel:
      self.w = np.zeros(n_features)
      for n in range(len(self.a)):
        self.w += self.a[n] * self.sv_y[n] * self.sv[n]
    else:
      self.w = None

  def predict(self, X):
    if self.w is not None:
      classification = np.dot(X,self.w) + self.b
      return np.sign(classification)

    else:
      y_predict = np.zeros(len(X))
      for i in range(len(X)):
        s = 0
        for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
          s += a * sv_y * self.kernel(X[i],sv)
      y_predict = s
      classification = y_predict + self.b
      return np.sign(classification)



      

# Support Vector Data Description

In [4]:
class SVDD():
  def __init__(self, kernel = gaussian_kernel, C = None, tao = 0, sigma = 5.0):
    self.kernel = kernel
    self.C = C
    self.tao = tao
    self.sigma = sigma
    if self.C is not None: self.C = float(self.C)
    

  def fit(self, X, y):
    n_samples, n_features = X.shape

    #Gram Matrix
    K = np.zeros((n_samples,n_samples))
    # for i in range(n_samples):
    #   for j in range(n_samples):
    #     K[i,j] = self.kernel(X[i], X[j])

    for i in range(n_samples):
      for j in range(n_samples):
        K[i,j] = (1-self.tao)*self.kernel(X[i], X[j], sigma = self.sigma) + self.tao*self.kernel(X[i], X[j], sigma = self.sigma*(1<<10))

    self.K = K
    P = cvxopt.matrix(K, tc='d')
    q = cvxopt.matrix(np.diagonal(K)*-1, tc='d')
    A = cvxopt.matrix(np.ones(n_samples), (1,n_samples) , tc='d')
    b = cvxopt.matrix(1.)

    if self.C is None:
      G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1), tc='d')
      h = cvxopt.matrix(np.zeros(n_samples), tc='d')
    else:
      tmp1 = np.diag(np.ones(n_samples) * -1)
      # tmp2 = np.identity(n_samples)
      tmp2 = np.ones(n_samples)
      G = cvxopt.matrix(np.vstack((tmp1, tmp2)), tc='d')
      tmp1 = np.zeros(n_samples)
      tmp2 = np.ones(n_samples) * self.C
      h = cvxopt.matrix(np.hstack((tmp1, tmp2)), tc='d')

    # solve QP problem
    solution = cvxopt.solvers.qp(P, q, G, h, A, b, verbose=True)

    #Lagrange Multipliers:
    a = np.ravel(solution['x'])

    #For support vectors a != 0
    sv = a > 1e-7
    ind = np.arange(len(a))[sv]
    self.a = a[sv]
    self.sv = X[sv]

    # compute the center of initial feature space
    center = np.dot(a.T, X)
    self.center = center


    '''
        compute the radius: eq(15)
        
        The distance from any support vector to the center of 
        the sphere is the hypersphere radius. 
        Here take the 1st support vector as an example.

        R^2 = K(xk, xk) -2(∑∝i)K(xi, xk) + ∑∑∝i∝jK(xi, xj)
        
    '''

    #Calculating Radius
    term_1 = K[ind[0], ind[0]]

    term_2 = -2*np.dot(K[ind[0], :], a)

    term_3 = np.dot(np.dot(a.T, K), a)

    radius = np.sqrt(term_1+term_2+term_3)

    self.term_3 = term_3
    self.radius = radius

    #number of SVs
    self.nSV = len(self.a)

    
    """
      Compute the distance of data points from the centre:
    
      d^2 = K(xi, xi) -2(∑∝j)K(xi, xj) + ∑∑∝j∝kK(xj, xk)

    """

    #Distance Matrix:
    D = np.zeros(n_samples)

    for i in range(n_samples):
      #term 1
      t_1 = K[i,i]
      
      #term 2
      t_2 = -2*np.dot(K[i,:], a)

      #term 3
      #same as term_3

      D[i] = np.sqrt(t_1 + t_2 + term_3)

    self.Distance = D




# KNN

In [5]:
class KNN():
  def __init__(self, kernel = linear_kernel, k=3):
      self.kernel = kernel
      self.k = k
  

  def fit(self, X, y):
    n_samples, n_features = X.shape
    #Gram Matrix
    K = np.zeros((n_samples,n_samples))
    for i in range(n_samples):
      for j in range(n_samples):
        K[i,j] = self.kernel(X[i], X[j])

    # d^2 = K[i,i] - 2K[i,j] + K[j,j]

    self.gram_matrix = K

    distance = np.zeros((n_samples,n_samples))
    prob = np.zeros(n_samples)
    
    for i in range(n_samples):
      for j in range(n_samples):
        distance[i, j] = K[i,i] - 2*K[i,j] + K[j,j]
      k_indices=np.argsort(distance[i])[:self.k+1]
      k_nearest_labels =[y[i] for i in k_indices]
      cnt = np.where(k_nearest_labels == y[i],1,0).sum() - 1
      prob[i]=cnt/self.k

    self.prob=prob


# ACFSVM

In [6]:
class ACFSVM():
  def __init__ (self, kernel = gaussian_kernel, C=1, k=5, rho = 1, gamma = 10, tao = 0, sigma = 5.0):
    self.kernel = kernel
    self.C = C
    self.k = k
    self.rho = rho
    self.gamma = gamma
    self.tao = tao
    self.sigma = sigma

  def fit(self, X, y):
    
    # positive are minority and negative are majority samples
    pIndex = np.where(y==1)
    nIndex = np.where(y==-1)

    n_samples, n_features = X.shape

    #Gram Matrix
    K = np.zeros((n_samples,n_samples))
    for i in range(n_samples):
      for j in range(n_samples):
        K[i,j] = (1-self.tao)*self.kernel(X[i], X[j], sigma = self.sigma) + self.tao*self.kernel(X[i], X[j], sigma = self.sigma*(1<<10))

    '''
    Calculating Class Probabilities
    
    (d^2)i,j = K[i,i] - 2K[i,j] + K[j,j]

    pi = num(own class)/k

    '''
    
    distance = np.zeros((n_samples,n_samples))
    prob = np.zeros(n_samples)
    
    for i in range(n_samples):
      for j in range(n_samples):
        distance[i, j] = K[i,i] - 2*K[i,j] + K[j,j]
      k_indices=np.argsort(distance[i])[:self.k+1]
      k_nearest_labels =[y[i] for i in k_indices]
      cnt = np.where(k_nearest_labels == y[i],1,0).sum() - 1
      prob[i]=cnt/self.k


    '''
    Calculating Affinity

    '''

    svdd = SVDD(kernel = self.kernel, C = self.C, tao = self.tao)
    svdd.fit(X[nIndex], y[nIndex])

    d_from_hyp = np.zeros(n_samples)
    d_from_hyp[nIndex] = svdd.Distance 
    d_max = max(svdd.Distance)
    d_min = min(svdd.Distance)

    trade_off = self.rho * svdd.radius

    affinity = np.zeros(n_samples)

    for i in range(n_samples):
      if y[i]==-1:
        if d_from_hyp[i] < trade_off:
          affinity[i] = (0.8*(d_from_hyp[i]-d_min)/(d_max-d_min))+0.2
        else:
          affinity[i] = 0.2*np.exp(self.gamma*(1-(d_from_hyp[i]/trade_off)))

      else:
        affinity[i] = 1


    '''
    Membership Value

    '''

    MV = np.ones(n_samples)

    for i in range(n_samples):
      if y[i]==-1:
        MV[i] = affinity[i]*prob[i]

    self.affinity = affinity
    self.MV = MV

    # r_imbalance = len(pIndex)/len(nIndex)

    '''
    Solving the Lagrangian

    '''

    P = cvxopt.matrix(np.outer(y,y) * K, tc='d')
    q = cvxopt.matrix(np.ones(n_samples) * -1, tc='d')
    A = cvxopt.matrix(y, (1,n_samples), tc='d')
    b = cvxopt.matrix(0.0)

    tmp1 = np.diag(np.ones(n_samples) * -1)
    # tmp2 = np.identity(n_samples)
    tmp2 = np.ones(n_samples)  #for l inf norm
    G = cvxopt.matrix(np.vstack((tmp1, tmp2)), tc='d')
    tmp1 = np.zeros(n_samples)
    tmp2 = MV * self.C
    h = cvxopt.matrix(np.hstack((tmp1, tmp2)), tc='d')

    # solve QP problem
    solution = cvxopt.solvers.qp(P, q, G, h, A, b, verbose=True)

    #Lagrange Multipliers:
    a = np.ravel(solution['x'])

    #For support vectors a != 0
    sv = a > 1e-7
    ind = np.arange(len(a))[sv]
    self.a = a[sv]
    self.sv = X[sv]
    self.sv_y = y[sv]
    print ("%d support vectors out of %d points" % (len(self.a), n_samples))

    #Intercept
    self.b = 0
    for n in np.arange(len(self.a)):
      self.b += self.sv_y[n]
      self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
    self.b /= len(self.a)
      


  def predict(self, X):
    y_predict = np.zeros(len(X))
    for i in range(len(X)):
      s = 0
      for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
        s += a * sv_y * ((1-self.tao)*self.kernel(X[i],sv, sigma=self.sigma) + self.tao*self.kernel(X[i],sv, sigma = self.sigma*(1<<10)))
      y_predict[i] = s
    classification = y_predict + self.b
    return np.sign(classification)



# Accuracy

In [7]:
import math
def accuracy(y_test,y_pred):
  tp,tn,fp,fn=(0,0,0,0)
  for i in range(len(y_pred)):
    if ((y_pred[i]==1) and (y_test[i]==1)):
      tp+=1
    elif ((y_pred[i]==-1) and (y_test[i]==1)):
      fn+=1
    elif ((y_pred[i]==1) and (y_test[i]==-1)):
      fp+=1
    else:
      tn+=1
  
  se=tp/(tp+fn)
  sp=tn/(tn+fp)
  if ((tp+fp)!=0):
    pr=tp/(tp+fp)
  else:
    pr=0 
  
  G = math.sqrt(se*sp)
  if (se+pr==0 or tp+fp==0):
    F=0
  else:
    F = 2*se*pr/(se+pr)
  return (G,F)